# Deliverable 2. Create a Customer Travel Destinations Map.
----
1. Create a folder called `Vacation_Search` to save all the files related with this deliverable.

2. Download the `Vacation_Search_starter_code.ipynb` Jupyter notebook,save it into your `Vacation_Search folder`, and rename it as `Vacation_Search.ipynb`.

3. In the `Vacation_Search.ipynb` file, ensure that the dependencies and the Google API key is imported correctly.

4. From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame named `city_data_df`.

5. Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

6. Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected. Name the DataFrame as `preferred_cities_df`.

7. Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

8. Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

9. Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

10. Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

11. Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

12. Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

13. Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

14. Review the formatting template provided that you'll use to add an information box to each marker in the map. In the pop-up for each city you'll add:

    1. The city name.

    2. The country code.

    3. The weather description and maximum temperature for the city.

15. Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

16. Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

17. Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

18. Take a screenshot of your map and save it to the `Vacation_Search` folder as `WeatherPy_vacation_map.png`.
---

## Ensure that the dependencies and the Google API key is imported correctly.

In [19]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import Google API key
from config import g_key

# Configure gmaps
gmaps.configure(api_key=g_key)

## From the `Weather_Database` folder you created in the "Deliverable 1," import the `WeatherPy_Database.csv` file as a Pandas DataFrame.

In [20]:
# Set the file path to import the WeatherPy_database.csv file
file_path = "../Weather_Database/WeatherPy_database.csv"

# Load the CSV file into a Pandas DataFrame
city_data_df = pd.read_csv(file_path)

# Display sample data
city_data_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Barrow,US,71.2906,-156.7887,30.22,80,100,10.36,overcast clouds
1,1,Harper,LR,4.3750,-7.7169,78.60,77,33,8.99,scattered clouds
2,2,Panipat,IN,29.3889,76.9681,77.90,73,0,2.93,clear sky
3,3,Nuevo Progreso,MX,23.1667,-97.9500,85.53,46,0,3.00,clear sky
4,4,Longyearbyen,SJ,78.2186,15.6401,39.04,93,100,3.44,fog
5,5,Ayan,RU,56.4500,138.1667,35.96,57,0,14.16,clear sky
6,6,Mataura,NZ,-46.1927,168.8643,30.49,99,9,2.08,clear sky
7,7,Rikitea,PF,-23.1203,-134.9692,73.53,78,88,15.70,light rain
8,8,Jawhar,SO,2.7809,45.5005,76.68,75,21,14.70,few clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,75.16,96,95,5.99,overcast clouds


## Write two input statements that prompt the user to enter their minimum and maximum temperature criteria for their vacation.

In [21]:
# Prompt the user to enter the minimum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

# Prompt the user to enter the maximum temperature criteria
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 50
What is the maximum temperature you would like for your trip? 80


## Create a new Pandas DataFrame by using the `loc` Pandas method to filter the `city_data_df` DataFrame for temperature criteria  collected.

In [22]:
# Filter the city_data_df DataFrame to find the cities that fit the criteria using the loc Pandas function
preferred_cities_df = city_data_df.loc[(city_data_df['Max Temp'] >= min_temp) & \
                                       (city_data_df['Max Temp'] <= max_temp)]
                                  
# Display sample data
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Harper,LR,4.3750,-7.7169,78.60,77,33,8.99,scattered clouds
2,2,Panipat,IN,29.3889,76.9681,77.90,73,0,2.93,clear sky
7,7,Rikitea,PF,-23.1203,-134.9692,73.53,78,88,15.70,light rain
8,8,Jawhar,SO,2.7809,45.5005,76.68,75,21,14.70,few clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,75.16,96,95,5.99,overcast clouds
10,10,Lebu,CL,-37.6167,-73.6500,56.82,76,100,13.51,overcast clouds
11,11,Mumford,GH,5.2618,-0.7590,76.55,87,42,11.54,scattered clouds
12,12,Castro,BR,-24.7911,-50.0119,57.58,88,100,6.78,moderate rain
14,14,Vaini,TO,-21.2000,-175.2000,71.76,83,40,9.22,scattered clouds
15,15,Lata,PT,40.1629,-8.3327,66.92,62,74,5.21,broken clouds


## Create a new Pandas DataFrame named `clean_travel_cities` by using the Pandas `dropna` function on the `preferred_cities_df` to drop any empty rows.

In [23]:
# Drop any empty rows in the preferred_cities_df DataFrame and create a new DataFrame.
clean_travel_cities = preferred_cities_df.dropna()

# Display sample data
clean_travel_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Harper,LR,4.3750,-7.7169,78.60,77,33,8.99,scattered clouds
2,2,Panipat,IN,29.3889,76.9681,77.90,73,0,2.93,clear sky
7,7,Rikitea,PF,-23.1203,-134.9692,73.53,78,88,15.70,light rain
8,8,Jawhar,SO,2.7809,45.5005,76.68,75,21,14.70,few clouds
9,9,Puerto Ayora,EC,-0.7393,-90.3518,75.16,96,95,5.99,overcast clouds
10,10,Lebu,CL,-37.6167,-73.6500,56.82,76,100,13.51,overcast clouds
11,11,Mumford,GH,5.2618,-0.7590,76.55,87,42,11.54,scattered clouds
12,12,Castro,BR,-24.7911,-50.0119,57.58,88,100,6.78,moderate rain
14,14,Vaini,TO,-21.2000,-175.2000,71.76,83,40,9.22,scattered clouds
15,15,Lata,PT,40.1629,-8.3327,66.92,62,74,5.21,broken clouds


## Use the `copy` Pandas function to create a new DataFrame, called `hotel_df`, by copying the following columns from the `clean_travel_cities` DataFrame: "City", "Country", "Max Temp", "Current Description", "Lat", "Lng".

In [24]:
# Create DataFrame called hotel_df by copying some columns from the clean_travel_cities DataFrame.
hotel_df = clean_travel_cities[["City","Country","Max Temp","Current Description", "Lat","Lng"]].copy()

# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng
1,Harper,LR,78.60,scattered clouds,4.3750,-7.7169
2,Panipat,IN,77.90,clear sky,29.3889,76.9681
7,Rikitea,PF,73.53,light rain,-23.1203,-134.9692
8,Jawhar,SO,76.68,few clouds,2.7809,45.5005
9,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518
10,Lebu,CL,56.82,overcast clouds,-37.6167,-73.6500
11,Mumford,GH,76.55,scattered clouds,5.2618,-0.7590
12,Castro,BR,57.58,moderate rain,-24.7911,-50.0119
14,Vaini,TO,71.76,scattered clouds,-21.2000,-175.2000
15,Lata,PT,66.92,broken clouds,40.1629,-8.3327


## Add a new empty column named `Hotel Name` to the `hotel_df` DataFrame.

In [25]:
# Add a new empty column, "Hotel Name", to the hotel_df DataFrame
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(10)


,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Harper,LR,78.60,scattered clouds,4.3750,-7.7169,
2,Panipat,IN,77.90,clear sky,29.3889,76.9681,
7,Rikitea,PF,73.53,light rain,-23.1203,-134.9692,
8,Jawhar,SO,76.68,few clouds,2.7809,45.5005,
9,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
10,Lebu,CL,56.82,overcast clouds,-37.6167,-73.6500,
11,Mumford,GH,76.55,scattered clouds,5.2618,-0.7590,
12,Castro,BR,57.58,moderate rain,-24.7911,-50.0119,
14,Vaini,TO,71.76,scattered clouds,-21.2000,-175.2000,
15,Lata,PT,66.92,broken clouds,40.1629,-8.3327,


## Review the hotel search parameters provided. These parameters are the same we used in this module; you'll use them to search for a hotel for each city.

In [26]:
# Review the parameters to search for a hotel
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}        

## Use a for loop to iterate through the `hotel_df` DataFrame, retrieve the latitude and longitude of each city to find the nearest hotel based on the search parameters provided, then add the hotel name to the `hotel_df` DataFrame. If a hotel isn't found, skip to the next city.

In [27]:
# Iterate through the hotel DataFrame 
for index, row in hotel_df.iterrows():
    # Fetch latitude and longitude from the DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude as parameters to the params dictionary
    params["location"] = f"{lat},{lng}"
    
    # Set up the base URL for the Google Directions API to get JSON data
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # Make an API request and retrieve the JSON data from the hotel search
    hotels = requests.get(base_url, params=params).json()
    
    # Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
# Display sample data
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Harper,LR,78.60,scattered clouds,4.3750,-7.7169,Screensaver
2,Panipat,IN,77.90,clear sky,29.3889,76.9681,SPOT ON 68644 Shri Shyam Guest House
7,Rikitea,PF,73.53,light rain,-23.1203,-134.9692,People ThankYou
8,Jawhar,SO,76.68,few clouds,2.7809,45.5005,Hotel Shabelle
9,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Lebu,CL,56.82,overcast clouds,-37.6167,-73.6500,Hostal Las Lilas
11,Mumford,GH,76.55,scattered clouds,5.2618,-0.7590,Peace Guest House
12,Castro,BR,57.58,moderate rain,-24.7911,-50.0119,CHACARA BAILLY
14,Vaini,TO,71.76,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
15,Lata,PT,66.92,broken clouds,40.1629,-8.3327,Casa Maquia


## Drop any rows in the `hotel_df` DataFrame where a hotel name is not found and store the resulting data into a new DataFrame named `clean_hotel_df`.

In [31]:
# Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how='any')

# Display sample data
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Harper,LR,78.60,scattered clouds,4.3750,-7.7169,Screensaver
2,Panipat,IN,77.90,clear sky,29.3889,76.9681,SPOT ON 68644 Shri Shyam Guest House
7,Rikitea,PF,73.53,light rain,-23.1203,-134.9692,People ThankYou
8,Jawhar,SO,76.68,few clouds,2.7809,45.5005,Hotel Shabelle
9,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
10,Lebu,CL,56.82,overcast clouds,-37.6167,-73.6500,Hostal Las Lilas
11,Mumford,GH,76.55,scattered clouds,5.2618,-0.7590,Peace Guest House
12,Castro,BR,57.58,moderate rain,-24.7911,-50.0119,CHACARA BAILLY
14,Vaini,TO,71.76,scattered clouds,-21.2000,-175.2000,Keleti Beach Resort
15,Lata,PT,66.92,broken clouds,40.1629,-8.3327,Casa Maquia


**Note:** If the resulting DataFrame is empty, uncomment the following code to load sample data into the `clean_hotel_df`.

In [32]:
# In case of an empty DataFrame, load the sample data provided
# clean_hotel_df = pd.read_csv("WeatherPy_vacation-backup.csv")
# clean_hotel_df.head(10)

## Create an CSV file to store the `clean_hotel_df` DataFrame as `WeatherPy_vacation.csv` in the Vacation_Search folder.

In [33]:
# Set the file name.
output_data_file = "WeatherPy_vacation.csv"

# Create a CSV file by using the clean_hotel_df DataFrame
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

### Review the formatting template provided that you'll use to add an information box to each marker in the map.

In the pop-up for each city you'll add: 

1. The city name

2. The country code

3. The weather description and maximum temperature for the city

In [34]:
# Review the formatting template provided
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

## Review the provided list comprehension code to retrieve the city data from each row, which will then be added to the formatting template and saved in the `hotel_info` list.

In [35]:
# Get the data from each row in the clean_hotel_df DataFrame, add it to the formatting template, and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# Display sample data
hotel_info[:10]


['\n<dl>\n<dt>Hotel Name</dt><dd>Screensaver</dd>\n<dt>City</dt><dd>Harper</dd>\n<dt>Country</dt><dd>LR</dd>\n<dt>Current Weather</dt><dd>scattered clouds and 78.6 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>SPOT ON 68644 Shri Shyam Guest House</dd>\n<dt>City</dt><dd>Panipat</dd>\n<dt>Country</dt><dd>IN</dd>\n<dt>Current Weather</dt><dd>clear sky and 77.9 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>People ThankYou</dd>\n<dt>City</dt><dd>Rikitea</dd>\n<dt>Country</dt><dd>PF</dd>\n<dt>Current Weather</dt><dd>light rain and 73.53 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hotel Shabelle</dd>\n<dt>City</dt><dd>Jawhar</dd>\n<dt>Country</dt><dd>SO</dd>\n<dt>Current Weather</dt><dd>few clouds and 76.68 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Finch Bay Galapagos Hotel</dd>\n<dt>City</dt><dd>Puerto Ayora</dd>\n<dt>Country</dt><dd>EC</dd>\n<dt>Current Weather</dt><dd>overcast clouds and 75.16 °F</dd>\n</dl>\n',
 '\n<dl>\n<dt>Hotel Name</dt><dd>Hostal Las Lila

## Use the provided code snippet to retrieve the latitude and longitude from each row and store them in a new DataFrame called `locations`.

In [36]:
# Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

# Display sample data
locations.head(10)

,Lat,Lng
1,4.3750,-7.7169
2,29.3889,76.9681
7,-23.1203,-134.9692
8,2.7809,45.5005
9,-0.7393,-90.3518
10,-37.6167,-73.6500
11,5.2618,-0.7590
12,-24.7911,-50.0119
14,-21.2000,-175.2000
15,40.1629,-8.3327


## Refactor your previous marker layer map code to create a marker layer map that will have pop-up markers for each city on the map.

In [42]:
# Add a marker layer for each city to the map. 
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

# Create a figure to add the Google map as a layer
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
fig.add_layer(marker_layer)

# Display the figure containing the map
fig

Figure(layout=FigureLayout(height='420px'))